In [ ]:
import numpy as np
from matplotlib import pyplot as plt

class NN:
    def init_weights(self):
        if self.y.shape[-1] == 1: # 1 output node only (i.e. binary classification);
            _weights1 = np.reshape(np.random.random((self.x.shape[1] + 1)*self.x.shape[1]), [self.x.shape[1] + 1, self.x.shape[1]])
            _weights2 = np.reshape(np.random.random((self.x.shape[1] + 1)*self.y.shape[1]), [self.x.shape[1] + 1, self.y.shape[1]])
        else: # more than 1 output node (i.e. multiple classification);
            _weights1 = np.reshape(np.random.random((self.x.shape[1] + 1)*self.y.shape[1]), [self.x.shape[1] + 1, self.y.shape[1]])
            _weights2 = np.reshape(np.random.random((self.y.shape[1] + 1)*self.y.shape[1]), [self.y.shape[1] + 1, self.y.shape[1]])
        return _weights1, _weights2

    def train(self, x, y, iterations, new_weights=False, learning_rate=1.0):
        '''
        x: must be a numpy array where columns are the features and rows are the samples;
        y: must be a numpy array where column(s) are the output labels and rows are the samples;
        '''
        self.x, self.y = x, y
        if not self.__dict__.__contains__('weights1') or new_weights:
            self.weights1, self.weights2 = self.init_weights()

        self.errors = []
        for _ in range(iterations):
            random_sample = np.random.randint(self.y.shape[0])
            input_layer = np.append(self.x[random_sample], np.ones(1), axis=0)
            input_layer.shape += (1,)

            hidden_layer = np.append(self.activation_function(self.weights1_multiplication(self.weights1, input_layer)), np.ones(1))
            hidden_layer.shape += (1,)
            output_layer = self.activation_function(self.weights2_multiplication(self.weights2, np.reshape(hidden_layer, [1, hidden_layer.shape[0]])))

            _error = self.y[random_sample] - output_layer
            self.errors.append(_error)

            feedback_weights2 = _error*output_layer*(1.0 - output_layer)*np.reshape(np.append(hidden_layer[:-1, 0], np.ones(1)), [hidden_layer.shape[0], 1])
            hidden_layer_error = self.weights2*_error*np.reshape(hidden_layer, [hidden_layer.shape[0], 1])*np.reshape(1.0 - hidden_layer, [hidden_layer.shape[0], 1])
            feedback_weights1 = hidden_layer_error[:-1, 0]*input_layer

            self.weights1 += feedback_weights1*learning_rate
            self.weights2 += feedback_weights2*learning_rate

    @staticmethod
    def activation_function(x):
        return 1.0 / (1.0 + np.exp(-x))

    @staticmethod
    def weights1_multiplication(x, y):
        _result = np.zeros((1, x.shape[1])).astype(float)
        for row in range(x.shape[0]):
            for col in range(x.shape[1]):
                _result[0, col] += x[row, col]*y[row]
        return _result[0]

    @staticmethod
    def weights2_multiplication(x, y):
        _result = np.zeros(x.shape[1]).astype(float)
        for row in range(x.shape[0]):
            for col in range(x.shape[1]):
                _result[col] += x[row, col]*y[0, row]
        return _result

    def plot_performance(self):
        def moving_average(y, moving_window=30):
            _y = []
            for _x in range(len(y)):
                _y.append(np.mean(y[:_x + 1]) if _x < moving_window else np.mean(y[_x - moving_window:_x]))
            return _y

        errors = [sum(_errors) for _errors in self.errors]
        fig = plt.figure(figsize=(15, 5))
        ax = plt.subplot(1, 3, 1)
        ax.plot(np.arange(0, len(errors), 1), errors, ls='-', lw=0.5, color=[1, 0, 0])
        ax.set_ylim(min(errors) - abs(0.1*min(errors)), max(errors) + 0.1*max(errors))
        ax.set_ylabel('Error', fontsize=15, fontweight='bold')
        ax.set_xlabel('Training Iterations', fontsize=15, fontweight='bold')
        for _axis in ['x', 'y']:
            ax.tick_params(axis=_axis, which='both', bottom='on', top=False, color='gray', labelcolor='gray')
        for _axis in ['top', 'right', 'bottom', 'left']:
            ax.spines[_axis].set_visible(False)
        ax = plt.subplot(1, 3, 2)
        zoomed_error_range = [0.5*np.mean(errors) - np.var(errors)**0.5, 0.5*np.mean(errors) + np.var(errors)**0.5]
        ax.plot(np.arange(0, len(errors), 1), errors, ls='-', lw=0.5, color=[1, 0, 0])
        ax.set_ylim(zoomed_error_range[0], zoomed_error_range[1])
        ax.set_xlabel('Training Iterations', fontsize=15, fontweight='bold')
        ax.set_title('[Zoomed in]', fontsize=15, fontweight='bold')
        for _axis in ['x', 'y']:
            ax.tick_params(axis=_axis, which='both', bottom='on', top=False, color='gray', labelcolor='gray')
        for _axis in ['top', 'right', 'bottom', 'left']:
            ax.spines[_axis].set_visible(False)
        ax = plt.subplot(1, 3, 3)
        _y = moving_average(errors)
        ax.plot(np.arange(0, len(_y), 1), _y, ls='-', lw=0.5, color=[1, 0, 0])
        ax.set_ylim(zoomed_error_range[0], zoomed_error_range[1])
        ax.set_xlabel('Training Iterations', fontsize=15, fontweight='bold')
        ax.set_title('[Smoothed & Zoomed in]', fontsize=15, fontweight='bold')
        for _axis in ['x', 'y']:
            ax.tick_params(axis=_axis, which='both', bottom='on', top=False, color='gray', labelcolor='gray')
        for _axis in ['top', 'right', 'bottom', 'left']:
            ax.spines[_axis].set_visible(False)
        fig.suptitle(f'{"Binary" if self.y.shape[-1] == 1 else "Multiple"}  Classification  Performance', fontsize=20, fontweight='bold')
        plt.show()

nn = NN()
nn.train(x, y, iterations=3000)
nn.plot_performance()